# This script scrapes Java-related Common Vulnerabilities and Exposures from the National Vulnerabilities Dataset

Importing the neccesary libraries.

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup

Pulling the dedicated page for JavaScript vulnerabilities from http://cve.mitre.org
Note: keyword = java

In [3]:
URL = "https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USC00457180,USC00390043&startDate=1941-12-07T09:31:20z&endDate=1945-07-12&format=json"

Requesting the URL above

In [4]:
page = requests.get(URL)

Specify a desired format for the “page” using the html parser in BeautifulSoup. This allows Python to read the various components of the page, rather than treating it as a long string.

In [5]:
formated_page = BeautifulSoup(page.text, "html.parser")

Printing the formatted page in a structured tree format (using the prettify method) for readability

In [17]:
print(formated_page.prettify())

[
{"DATE":"1941-12-07","STATION":"USC00457180","TMAX":"   72","TMIN":"   -6","PRCP":"    0","TOBS":"   28"}
,{"DATE":"1941-12-08","STATION":"USC00457180","TMAX":"   72","TMIN":"   -6","PRCP":"    0","TOBS":"   17"}
,{"DATE":"1941-12-09","STATION":"USC00457180","TMAX":"   22","TMIN":"  -28","PRCP":"    0","TOBS":"    6"}
,{"DATE":"1941-12-10","STATION":"USC00457180","TMAX":"   11","TMIN":"  -28","PRCP":"    0","TOBS":"  -17"}
,{"DATE":"1941-12-11","STATION":"USC00457180","SNOW":"   15","TMAX":"  -11","TMIN":"  -28","PRCP":"   15","TOBS":"  -11","SNWD":"    0"}
,{"DATE":"1941-12-12","STATION":"USC00457180","TMAX":"    6","TMIN":"  -44","PRCP":"    0","TOBS":"  -11"}
,{"DATE":"1941-12-13","STATION":"USC00457180","TMAX":"   28","TMIN":"  -33","PRCP":"    0","TOBS":"   22","SNWD":"    0"}
,{"DATE":"1941-12-14","STATION":"USC00457180","TMAX":"   61","TMIN":"   11","PRCP":"   15","TOBS":"   17"}
,{"DATE":"1941-12-15","STATION":"USC00457180","TMAX":"   89","TMIN":"    6","PRCP":"   43","TOBS":

This is my attempt to convert to dataframe, but still having errors

In [18]:
import pandas as pd

In [20]:
climate_data_frame = pd.DataFrame(formated_page)

KeyError: 0

In [21]:
climate_data_frame = pd.Series(formated_page)

KeyError: 0

A function to extract the CVE ids

In [6]:
def extract_CVE_ids(formated_page):
    CVE_ids =[]
    for table in formated_page.find_all(name="div", attrs={"id":"TableWithRules"}):
        for row in table.find_all(name="tr"):
            for column in row.find_all(name="td", attrs={"valign":"top", "nowrap":"nowrap"}):
                for ids in column.find_all(name="a"):
                    CVE_ids.append(ids.string)
    return CVE_ids

Calling the extract_CVE_ids function

In [7]:
CVE_ids_list = extract_CVE_ids(formated_page)

A function to filter record in the extract_CVE_description function. The function is passed as an argument in one of its .find_all 

In [8]:
def has_valign_but_no_wrap(tag):
    return tag.has_attr('valign') and not tag.has_attr('nowrap') and not tag.has_attr('style')

Implementation of extract_CVE_description function

Importing the regular expression module to treat the vulnerability description string

In [9]:
import re

In [10]:
def extract_CVE_description(formated_page):
    CVE_descr =[]
    for table in formated_page.find_all(name="div", attrs={"id":"TableWithRules"}):
        for row in table.find_all(name="tr"):
            for descr in row.find_all(has_valign_but_no_wrap):
                delimited_descr = re.split("\n+", descr.string)
                CVE_descr.append(delimited_descr[0])
    return CVE_descr

Calling the extract_CVE_description function

In [11]:
CVE_description_list = extract_CVE_description(formated_page)

Building frame for the extracted data

Import required library

In [12]:
import pandas as pd

In [21]:
data_frame = pd.DataFrame(list(zip(CVE_ids_list,CVE_description_list)),columns=["CVE","vulnerability_description"])

In [22]:
data_frame.head()

,CVE,vulnerability_description
0,CVE-2021-25325,MISP 2.4.136 has XSS via galaxy cluster elemen...
1,CVE-2021-23935,OX App Suite through 7.10.4 allows XSS via an ...
2,CVE-2021-23934,OX App Suite through 7.10.4 allows XSS via a c...
3,CVE-2021-23933,OX App Suite through 7.10.4 allows XSS via Jav...
4,CVE-2021-22849,Hyweb HyCMS-J1 backend editing function does n...


In [25]:
data_frame.to_csv(r"C:\Users\Semiu\Documents\java-codesecurity\JavaVulData\JavaVulData.csv", encoding='utf-8')